# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
!pip install razdel

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

from razdel import tokenize

In [3]:
data = pd.read_csv('labeled.csv')

In [4]:
data.head(10)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [5]:
train, test = train_test_split(data, test_size=0.3, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [6]:
def get_token(text):
    tokens = list(tokenize(text))
    return [token.text for token in tokens]

In [7]:
default_vectorizer = CountVectorizer()
razdel_vectorizer = CountVectorizer(tokenizer=get_token)

In [8]:
def get_X_by_vectorizer(comments, vectorizer, is_transform=True):
    if is_transform:
        return vectorizer.fit_transform(comments)
    return vectorizer.transform(comments)

In [9]:
def compute_model_quality(name_vectorizer, vectorizer, model, train, test):
    print(f'Векторайзер {name_vectorizer}')
    X = get_X_by_vectorizer(train.comment, vectorizer)
    y = train.toxic.values

    X_test = get_X_by_vectorizer(test.comment, vectorizer, False)
    y_test = test.toxic.values
    
    model.fit(X, y)
    
    preds = model.predict(X_test)
    print(classification_report(y_test, preds, zero_division=0))
    print(30*'-')

In [10]:
vectorizers = [('default', default_vectorizer), ('razdel', razdel_vectorizer)]
for i in range(len(vectorizers)):
    name_vectorizer = vectorizers[i][0]
    vectorizer = vectorizers[i][1]
    compute_model_quality(name_vectorizer, vectorizer, LogisticRegression(), train, test)

Векторайзер default


C:\Users\adugeen\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90      2878
         1.0       0.85      0.69      0.76      1446

    accuracy                           0.86      4324
   macro avg       0.86      0.82      0.83      4324
weighted avg       0.86      0.86      0.85      4324

------------------------------
Векторайзер razdel
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90      2878
         1.0       0.85      0.72      0.78      1446

    accuracy                           0.86      4324
   macro avg       0.86      0.83      0.84      4324
weighted avg       0.86      0.86      0.86      4324

------------------------------


C:\Users\adugeen\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


В целом в обоих случаях векторайзеры показывают одинаковые результаты, но стоит отметить, что иногда модель на векторайзере на разделе получает чуть лучшие результаты, чем на дефолтном.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [11]:
import numpy as np

In [12]:
data_table = np.array([
                [1, 1, 1, 0, 0, 0],
                [1, 1, 1, 0, 0, 0],
                [3, 0, 1, 1, 0, 0],
                [1, 0, 0, 1, 1, 0],
                [0, 0, 0, 0, 0, 1]])

In [13]:
N = data_table.shape[0]
result = []

for sent in data_table:
    norm_sent = []
    for word in range(len(sent)):
        norm_sent.append(sent[word] / sent.sum() * np.log10(N / np.count_nonzero(data_table.T[word])))
    result.append(norm_sent)
    
result_table = pd.DataFrame(data=result, columns=['я', 'ты', 'и', 'только', 'не', 'он'], 
                           index=['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он'])
result_table

,я,ты,и,только,не,он
я и ты,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
ты и я,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
"я, я и только я",0.058146,0.000000,0.04437,0.079588,0.00000,0.00000
только не я,0.032303,0.000000,0.00000,0.132647,0.23299,0.00000
он,0.000000,0.000000,0.00000,0.000000,0.00000,0.69897


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [14]:
import nltk
from nltk.corpus import stopwords
from pprint import pprint

In [15]:
def get_init_stopwords():
    sw = stopwords.words('russian')
    sw.extend(['это', 'например', 'тебе'])
    return sw

In [16]:
def process_models(name_vectorizer, name_model, vectorizer, model, train, test):
    print(f'Модель {name_model} с векторайзером {name_vectorizer}')
    
    X = get_X_by_vectorizer(train.clear_comment, vectorizer)
    y = train.toxic.values

    X_test = get_X_by_vectorizer(test.clear_comment, vectorizer, False)
    y_test = test.toxic.values
    
    model.fit(X, y)
    
    preds = model.predict(X_test)
    print('Качество модели')
    print(classification_report(y_test, preds, zero_division=0))
    
    probas = np.array(model.predict_proba(X_test))[:, 1]
    sorted_comments = sorted(zip(probas, test.clear_comment), reverse=True)

    print('Самые токсичные тексты')
    pprint([x[1] for x in sorted_comments][:10])
    print(40*'-')

In [17]:
max_df = 0.6
min_df = 2
ngram_range = (1, 2)

count_vectorizer = CountVectorizer(stop_words=get_init_stopwords(), 
                                   max_df=max_df, min_df=min_df, 
                                   tokenizer=get_token, ngram_range=ngram_range)
tfidf_vectorizer = TfidfVectorizer(tokenizer=get_token, 
                                   stop_words=get_init_stopwords(), 
                                   ngram_range=ngram_range, 
                                   max_df=max_df, min_df=min_df,)

In [18]:
train['clear_comment'] = train.comment.str.replace('[^\w\s]', '')
test['clear_comment'] = test.comment.str.replace('[^\w\s]', '')

C:\Users\adugeen\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\adugeen\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
vectorizers = [('CountVectorizer', count_vectorizer), ('TfidfVectorizer', tfidf_vectorizer)]
models = [('DecisionTreeClassifier', DecisionTreeClassifier(criterion='entropy', min_samples_split=3, 
                                                            min_samples_leaf=3, 
                                                            max_depth=10000, max_leaf_nodes=10000)),
          ('RandomForestClassifier', RandomForestClassifier(criterion='entropy', n_estimators=500))]
               
for i in range(len(vectorizers)):
    name_model = models[i][0]
    model = models[i][1]
    name_vectorizer = vectorizers[i][0]
    vectorizer = vectorizers[i][1]
    process_models(name_vectorizer, name_model, vectorizer, model, train, test)

Модель DecisionTreeClassifier с векторайзером CountVectorizer
Качество модели
              precision    recall  f1-score   support

         0.0       0.83      0.81      0.82      2878
         1.0       0.64      0.66      0.65      1446

    accuracy                           0.76      4324
   macro avg       0.74      0.74      0.74      4324
weighted avg       0.77      0.76      0.77      4324

Самые токсичные тексты
['чувак я хоть и не особый любитель каверов да и всей теперешной музыкикоторую '
 'я уже не слушаю ты при своём исполнении system of a dawn  Chop Suey просто '
 'понравился твоим русским исполнением  плюсярю тебе бро\n',
 'чем больше бояр тем меньше холопов на одного барина какая странная '
 'математика следовательно старшие бояре будут чморить младших бояр так как '
 'холопов мало\n',
 'хуйца пососи блядина лахтинская иди на завод сосать хуй за 15К под твои '
 'потребности как раз\n',
 'хотят завезти Епта они и так приезжают И даже больше Ну даст им вова паспорт '


Во-первых, как можно заметить, случайный лес работает намного лучше, чем дерево решений с кучей настроенных вручную параметров. Возможно это из-за используемого векторайзера, но я думаю, что это даёт всё равно небольшой прирост. Во-вторых, в случае с деревом в самые токсичные тексты попадают и не совсем токсичные. В случае со случайным лесом таких ситуаций намного меньше (все тексты токсичные из 10 самых токсичных). В-третьих, полноценно тексты всё же отличаются между собой, однако среди них есть общее, которое выражается в обильном использовании различных ругательств, что маркирует тексты как токсичные.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [20]:
def show_most_informative_features(vectorizer, clf, n=5):
    feature_names = vectorizer.get_feature_names()

    coefs_with_fns = sorted(zip(clf.coef_[0] if type(clf) in 
                                [LogisticRegression, MultinomialNB] else clf.feature_importances_, feature_names))

    top = coefs_with_fns[:-(n + 1):-1]
    for (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s" % (coef_2, fn_2))

In [21]:
classifiers = [
                LogisticRegression(),
                MultinomialNB(),
                RandomForestClassifier(),
                DecisionTreeClassifier()]
vectorizers = [
                count_vectorizer,
                tfidf_vectorizer]

In [22]:
for vectorizer in vectorizers:
    print(20*'-' + vectorizer.__class__.__name__ + 20*'-')
    
    X = get_X_by_vectorizer(train.clear_comment, vectorizer)
    y = train.toxic.values

    X_test = get_X_by_vectorizer(test.clear_comment, vectorizer, False)
    y_test = test.toxic.values
    
    for classifier in classifiers:
        print(20*'-' + classifier.__class__.__name__ + 20*'-')
        
        classifier.fit(X, y)
        show_most_informative_features(vectorizer, classifier)

--------------------CountVectorizer--------------------
--------------------LogisticRegression--------------------
	2.7975	хохлов         
	2.7084	хохлы          
	2.1199	нахуй          
	2.1053	дебил          
	1.9164	тупые          
--------------------MultinomialNB--------------------
	-5.6125	просто         
	-6.2037	всё            
	-6.3186	вообще         
	-6.3361	почему         
	-6.4096	ещё            
--------------------RandomForestClassifier--------------------
	0.0092	хохлы          
	0.0075	хохлов         
	0.0070	нахуй          
	0.0060	очень          
	0.0043	быдло          
--------------------DecisionTreeClassifier--------------------
	0.0165	хохлы          
	0.0132	нахуй          
	0.0118	хохлов         
	0.0088	очень          
	0.0070	блядь          
--------------------TfidfVectorizer--------------------
--------------------LogisticRegression--------------------
	4.2503	хохлы          
	3.9383	хохлов         
	3.7292	нахуй          
	2.8453	пиздец         
	2.7694	б

По результатам можно заключить следующее. TfidfVectorizer работает намного лучше, чем CountVectorizer. Логистическая регрессия показывает лучшие результаты по сравнению с другими классификаторами. Байесовский алгоритм показывает наихудшие результаты. В целом у каждого классификатора результаты одинаковы. Самые токсичные слова - это как раз те, которые обозначают какое-то ругательное слово, мат.